<a href="https://colab.research.google.com/github/veren4/SMILES_featurization/blob/master/My_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Tutorial](https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/)

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pickle

infile1 = open('/content/drive/My Drive/Rostlab internship/7_PyTorch/Tokenized_Dataset', 'rb')
tokenized_dataset = pickle.load(infile1)
infile1.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###First Trial

In [2]:
import torch
import torch.nn as nn

In [3]:
input_dim = 5 #3         # size of the input at each step
hidden_dim = 10 #3       # size of the hidden state and cell state at each step
n_layers = 1          # number of LSTM layers stacked on top of each other

lstm_layer = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)

In [4]:
batch_size = 1
seq_len = 1

#inp = torch.randn(batch_size, seq_len, input_dim)
hidden_state = torch.randn(n_layers, batch_size, hidden_dim)
cell_state = torch.randn(n_layers, batch_size, hidden_dim)
hidden = (hidden_state, cell_state)    # tuple

In [5]:
type(inp)

torch.Tensor

In [6]:
print("Input:", inp.size())
print("Hidden state:", hidden_state.size())
print("Cell state:", cell_state.size())

Input: torch.Size([1, 1, 5])
Hidden state: torch.Size([1, 1, 10])
Cell state: torch.Size([1, 1, 10])


Now let's see what comes out of the LSTM layer:

In [7]:
out, hidden = lstm_layer(inp, hidden)
print("Output shape: ", out.shape)
print("Hidden: ", hidden)

Output shape:  torch.Size([1, 1, 10])
Hidden:  (tensor([[[-0.4495,  0.1912,  0.1779,  0.4032, -0.1397,  0.3387, -0.1074,
          -0.4542, -0.2520,  0.3526]]], grad_fn=<StackBackward>), tensor([[[-0.6424,  0.3893,  0.2377,  1.1927, -0.2825,  0.8477, -0.4810,
          -0.7275, -0.5518,  0.7644]]], grad_fn=<StackBackward>))


##Data preparation (int_tokens)

####Create vocabulary

In [94]:
#len(tokenized_dataset)

14

In [2]:
alphabet = set()

for i in tokenized_dataset:
  alphabet.update(i)

In [3]:
alphabet    # length: 12

{'(', ')', '1', '2', '3', '4', '=', 'C', 'N', 'O', 'P', 'S'}

####Create indices to vectorize words

In [6]:
dict_token_alphabet = {}
dict_token_alphabet.update({'Padding': 0, 'Unknown': 1})

index = 2
for i in alphabet:
  dict_token_alphabet.update({i: index})
  index = index+1

dict_token_alphabet

{'(': 9,
 ')': 13,
 '1': 5,
 '2': 10,
 '3': 4,
 '4': 7,
 '=': 6,
 'C': 8,
 'N': 11,
 'O': 12,
 'P': 3,
 'Padding': 0,
 'S': 2,
 'Unknown': 1}

####Vectorize SMILES

In [ ]:
#tokenized_dataset[1]

In [7]:
int_tokens = [None]*len(tokenized_dataset)    # empty list of length 14

for i in range(len(tokenized_dataset)):
  int_tokens[i] = [None]*len(tokenized_dataset[i])

  for j in range(len(tokenized_dataset[i])):
    int_tokens[i][j] = dict_token_alphabet.get(tokenized_dataset[i][j])

In [8]:
#int_tokens[1]

[8, 8, 8, 9, 8, 13, 8, 9, 6, 12, 13, 8, 9, 6, 12, 13, 12]

####Batching + Padding

[Tutorial](https://github.com/chrisvdweth/ml-toolkit/blob/master/pytorch/notebooks/minimal-example-lstm-input.ipynb)\
Still open: What is a batch for me? For now, I will go with 3 SMILES.\
The sequence length has to be constant within 1 batch. -> Pad accordingly\
Choose Batches with equal length so that I have to do minimal Padding.

In [36]:
batch = [int_tokens[0], int_tokens[1], int_tokens[3]] 

pad_to_this = len(max(batch, key=len))    #  length of the longest list in the batch -> All other elements need to be padded to this length

In [37]:
pad_beginning = True

for SMILES in batch:
  while(len(SMILES) < pad_to_this):
    if(pad_beginning == True):
      SMILES.insert(0, 0)
      pad_beginning = !pad_beginning
    else:
      SMILES.append(0)
      pad_beginning = !pad_beginning

In [39]:
#for SMILES in batch:
#  print(len(SMILES))

In [44]:
float_batch = [[float(token) for token in SMILES] for SMILES in batch]

##Putting my data into a model

####Setup

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [10]:
import platform
print('Using python: ', platform.python_version())
print('Using torch version: ', torch.__version__)
print('Using device: ', device)
# Machine: 2015 13" Macbook Pro, i5 dual core

import torch.nn as nn
import timeit

Using python:  3.6.9
Using torch version:  1.6.0+cu101
Using device:  cpu


####Model

In [47]:
# choose the input parameters
input_size = 33         # max length of SMILES (token sequence)
hidden_dim = 1   # What does this parameter do?
seq_len= 2  # What does this parameter do?

# define the model
pytorch_lstm=nn.LSTM(input_size, hidden_dim)

# initialise the lstm
for p in pytorch_lstm.parameters():
    nn.init.constant_(p, val=0.3)

In [51]:
#X = torch.randn(seq_len, 1, input_size)
X = torch.tensor([float_batch])
X

tensor([[[ 8.,  9.,  8.,  9.,  8.,  9.,  8., 12.,  3.,  9.,  6., 12., 13.,  9.,
          12., 13., 12., 13., 12., 13., 12., 13.,  8.,  9.,  6., 12., 13.,  8.,
           9.,  6., 12., 13., 12.],
         [ 0.,  0.,  8.,  8.,  8.,  9.,  8., 13.,  8.,  9.,  6., 12., 13.,  8.,
           9.,  6., 12., 13., 12.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.],
         [ 8.,  5.,  6.,  8.,  8.,  9.,  6.,  8.,  8.,  9.,  6.,  8.,  5., 13.,
          12., 13.,  8.,  8.,  8.,  9.,  6., 12., 13., 12.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.]]])

In [49]:
output_pytorch, (h_pytorch, _) = pytorch_lstm(X)

In [52]:
runs=10**4

print("Time Pytorch LSTM {} runs: {:.3f}s".format(runs, timeit.timeit("pytorch_lstm(X)", 
                                       setup="from __main__ import pytorch_lstm, X", 
                                       number=runs))
     )

Time Pytorch LSTM 10000 runs: 1.476s


In [16]:
#X.size()

torch.Size([1, 1, 17])